In [8]:
from openai import OpenAI
from dotenv import load_dotenv
import os

# ✅ Charge la clé depuis le fichier .env
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")

# ✅ Passe explicitement la clé à l’objet OpenAI
client = OpenAI(api_key=api_key)


In [9]:
prompt = "Combien y a-t-il de protéines dans 100g de steak ? Donne une réponse simple et directe en grammes."

try:
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    print("✅ Réponse de ChatGPT :")
    print(response.choices[0].message.content)
except Exception as e:
    print("❌ Erreur avec l'API :", e)


✅ Réponse de ChatGPT :
Il y a environ 20-25g de protéines dans 100g de steak.


In [11]:
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv
from tqdm import tqdm

# Charger la clé API depuis le fichier .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Lire le fichier CSV avec encodage
df = pd.read_csv("data/food/food.csv", encoding="ISO-8859-1")

# Colonnes à compléter
nutrients = [
    "calories", "Protéines", "glucides", "dont_sucres",
    "lipides", "dont_acides_gras_satures", "fibres", "sel", "eau"
]

# Fonction qui appelle l'API pour un aliment donné
def get_nutrition(aliment, poids):
    prompt = (
        f"Peux-tu me donner les valeurs nutritionnelles moyennes pour {poids}g de {aliment} "
        f"(en format dictionnaire Python) avec les champs suivants : "
        f"calories(kcal), Protéines(g), glucides(g), dont_sucres(g), lipides(g), dont_acides_gras_satures(g), fibres(g), sel(g), eau(ml)."
    )
    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        content = response.choices[0].message.content
        data = eval(content)  # ⚠️ Assuré uniquement si on fait confiance à la réponse GPT
        return data
    except Exception as e:
        print(f"Erreur pour {aliment} ({poids}g) : {e}")
        return {}

# Boucle sur chaque ligne pour compléter si valeurs manquantes
for idx, row in tqdm(df.iterrows(), total=len(df)):
    aliment = row["aliments"]
    try:
        poids_unitaire = float(row["poid aliments(gr)"])
        quantite = float(row["quantite"])
        poids_total = int(poids_unitaire * quantite)
    except:
        continue  # Skip si données invalides

    # Ne pas refaire si calories déjà présentes
    if not pd.isnull(row["calories(kcal)"]):
        continue

    result = get_nutrition(aliment, poids_total)
    for nut in nutrients:
        if nut in result:
            df.at[idx, nut] = result[nut]

# Sauvegarde du nouveau fichier
df.to_csv("food_completed.csv", index=False)
print("✅ Données nutritionnelles complétées et sauvegardées dans food_completed.csv")


  0%|          | 0/10 [00:00<?, ?it/s]

100%|██████████| 10/10 [00:04<00:00,  2.37it/s]

✅ Données nutritionnelles complétées et sauvegardées dans food_completed.csv


In [13]:
from openai import OpenAI
import pandas as pd
import os
from dotenv import load_dotenv
from tqdm import tqdm

# Chargement de la clé API
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

# Chargement du CSV
df = pd.read_csv("data/food/food.csv", encoding="ISO-8859-1")

# Colonnes nutritionnelles à compléter
nutrients = [
    "calories", "Protéines", "glucides", "dont_sucres",
    "lipides", "dont_acides_gras_satures", "fibres", "sel", "eau"
]

def get_nutrition(aliment, quantite=None, poids=None):
    # Vérifie proprement si les valeurs ne sont pas NaN
    has_poids = pd.notnull(poids)
    has_quantite = pd.notnull(quantite)

    if has_poids and has_quantite:
        description = f"{int(poids * quantite)}g de {aliment}"
    elif has_poids and not has_quantite:
        description = f"{int(poids)}g de {aliment}"
    elif has_quantite and not has_poids:
        description = f"{int(quantite)} {aliment}"  # ex : 2 œufs
    else:
        return {}

    prompt = (
        f"Donne les valeurs nutritionnelles moyennes pour {description} "
        f"sous forme de dictionnaire Python avec ces clés : "
        f"calories (kcal), Protéines (g), glucides (g), dont_sucres (g), lipides (g), "
        f"dont_acides_gras_satures (g), fibres (g), sel (g), eau (ml)."
    )

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": prompt}],
            temperature=0.3
        )
        reply = response.choices[0].message.content
        return eval(reply)
    except Exception as e:
        print(f"❌ Erreur pour {description} : {e}")
        return {}


# Remplissage du DataFrame
for idx, row in tqdm(df.iterrows(), total=len(df)):
    if not pd.isnull(row["calories(kcal)"]):
        continue

    aliment = row["aliments"]

    try:
        poids = float(row.get("poid aliments(gr)", None))
    except:
        poids = None

    try:
        quantite = float(row.get("quantite", None))
    except:
        quantite = None

    # Skip si les deux sont manquants
    if poids is None and quantite is None:
        print(f"⏭️ Ignoré : pas d'infos sur '{aliment}' (ligne {idx})")
        continue

    result = get_nutrition(aliment, quantite=quantite, poids=poids)

    for nut in nutrients:
        if nut in result:
            df.at[idx, nut] = result[nut]

# Sauvegarde finale
df.to_csv("food_completed.csv", index=False)
print("✅ Fichier complété avec succès : food_completed.csv")

100%|██████████| 10/10 [00:13<00:00,  1.40s/it]

✅ Fichier complété avec succès : food_completed.csv
